### Code Convergence
This notebook illustrates convergence to terminal state through code. Here action set, reward set and state set has been defined. The Action set is a list of the format [Row_Index, Index of 1st Number, Index of 2nd Number] and it represents swapping of the values present at the index location of that row. This action results in change of invalid elements in the sudoku grid thus resulting in state change. Reward is associated to each action depending on the difference between the valid elements and invalid elements.  Incase of an illegal action which is related to swapping of provided numbers in the sudoku puzzle, the reward assocaited would be -1000 with no change in state.

The code at each step would execute an action which results in highest reward. Incase of multiple actions having the same reward value, random choice of action is performed. If the next action to be executed is equal to the previous action then a  check is performed to select another action with the same reward if available else select any other action with a positive reward value. If no action is available then the loop would break. For each run, the number of iterations are limited to 2000 and the loop is terminated on convergence i.e. when terminal state reached - 0 invalids

In [1]:
import ast
from numpy import random
import copy

#### Action Set

In [2]:
actions_dict={0:[0,0,1],1:[0,0,2],2:[0,0,3],3:[0,0,4],4:[0,0,5],5:[0,0,6],6:[0,0,7],7:[0,0,8],8:[0,1,2],9:[0,1,3],10:[0,1,4],11:[0,1,5],12:[0,1,6],13:[0,1,7],14:[0,1,8],15:[0,2,3],16:[0,2,4],17:[0,2,5],18:[0,2,6],19:[0,2,7],20:[0,2,8],21:[0,3,4],22:[0,3,5],23:[0,3,6],24:[0,3,7],25:[0,3,8],26:[0,4,5],27:[0,4,6],28:[0,4,7],29:[0,4,8],30:[0,5,6],31:[0,5,7],32:[0,5,8],33:[0,6,7],34:[0,6,8],35:[0,7,8],36:[1,0,1],37:[1,0,2],38:[1,0,3],39:[1,0,4],40:[1,0,5],41:[1,0,6],42:[1,0,7],43:[1,0,8],44:[1,1,2],45:[1,1,3],46:[1,1,4],47:[1,1,5],48:[1,1,6],49:[1,1,7],50:[1,1,8],51:[1,2,3],52:[1,2,4],53:[1,2,5],54:[1,2,6],55:[1,2,7],56:[1,2,8],57:[1,3,4],58:[1,3,5],59:[1,3,6],60:[1,3,7],61:[1,3,8],62:[1,4,5],63:[1,4,6],64:[1,4,7],65:[1,4,8],66:[1,5,6],67:[1,5,7],68:[1,5,8],69:[1,6,7],70:[1,6,8],71:[1,7,8],72:[2,0,1],73:[2,0,2],74:[2,0,3],75:[2,0,4],76:[2,0,5],77:[2,0,6],78:[2,0,7],79:[2,0,8],80:[2,1,2],81:[2,1,3],82:[2,1,4],83:[2,1,5],84:[2,1,6],85:[2,1,7],86:[2,1,8],87:[2,2,3],88:[2,2,4],89:[2,2,5],90:[2,2,6],91:[2,2,7],92:[2,2,8],93:[2,3,4],94:[2,3,5],95:[2,3,6],96:[2,3,7],97:[2,3,8],98:[2,4,5],99:[2,4,6],100:[2,4,7],101:[2,4,8],102:[2,5,6],103:[2,5,7],104:[2,5,8],105:[2,6,7],106:[2,6,8],107:[2,7,8],108:[3,0,1],109:[3,0,2],110:[3,0,3],111:[3,0,4],112:[3,0,5],113:[3,0,6],114:[3,0,7],115:[3,0,8],116:[3,1,2],117:[3,1,3],118:[3,1,4],119:[3,1,5],120:[3,1,6],121:[3,1,7],122:[3,1,8],123:[3,2,3],124:[3,2,4],125:[3,2,5],126:[3,2,6],127:[3,2,7],128:[3,2,8],129:[3,3,4],130:[3,3,5],131:[3,3,6],132:[3,3,7],133:[3,3,8],134:[3,4,5],135:[3,4,6],136:[3,4,7],137:[3,4,8],138:[3,5,6],139:[3,5,7],140:[3,5,8],141:[3,6,7],142:[3,6,8],143:[3,7,8],144:[4,0,1],145:[4,0,2],146:[4,0,3],147:[4,0,4],148:[4,0,5],149:[4,0,6],150:[4,0,7],151:[4,0,8],152:[4,1,2],153:[4,1,3],154:[4,1,4],155:[4,1,5],156:[4,1,6],157:[4,1,7],158:[4,1,8],159:[4,2,3],160:[4,2,4],161:[4,2,5],162:[4,2,6],163:[4,2,7],164:[4,2,8],165:[4,3,4],166:[4,3,5],167:[4,3,6],168:[4,3,7],169:[4,3,8],170:[4,4,5],171:[4,4,6],172:[4,4,7],173:[4,4,8],174:[4,5,6],175:[4,5,7],176:[4,5,8],177:[4,6,7],178:[4,6,8],179:[4,7,8],180:[5,0,1],181:[5,0,2],182:[5,0,3],183:[5,0,4],184:[5,0,5],185:[5,0,6],186:[5,0,7],187:[5,0,8],188:[5,1,2],189:[5,1,3],190:[5,1,4],191:[5,1,5],192:[5,1,6],193:[5,1,7],194:[5,1,8],195:[5,2,3],196:[5,2,4],197:[5,2,5],198:[5,2,6],199:[5,2,7],200:[5,2,8],201:[5,3,4],202:[5,3,5],203:[5,3,6],204:[5,3,7],205:[5,3,8],206:[5,4,5],207:[5,4,6],208:[5,4,7],209:[5,4,8],210:[5,5,6],211:[5,5,7],212:[5,5,8],213:[5,6,7],214:[5,6,8],215:[5,7,8],216:[6,0,1],217:[6,0,2],218:[6,0,3],219:[6,0,4],220:[6,0,5],221:[6,0,6],222:[6,0,7],223:[6,0,8],224:[6,1,2],225:[6,1,3],226:[6,1,4],227:[6,1,5],228:[6,1,6],229:[6,1,7],230:[6,1,8],231:[6,2,3],232:[6,2,4],233:[6,2,5],234:[6,2,6],235:[6,2,7],236:[6,2,8],237:[6,3,4],238:[6,3,5],239:[6,3,6],240:[6,3,7],241:[6,3,8],242:[6,4,5],243:[6,4,6],244:[6,4,7],245:[6,4,8],246:[6,5,6],247:[6,5,7],248:[6,5,8],249:[6,6,7],250:[6,6,8],251:[6,7,8],252:[7,0,1],253:[7,0,2],254:[7,0,3],255:[7,0,4],256:[7,0,5],257:[7,0,6],258:[7,0,7],259:[7,0,8],260:[7,1,2],261:[7,1,3],262:[7,1,4],263:[7,1,5],264:[7,1,6],265:[7,1,7],266:[7,1,8],267:[7,2,3],268:[7,2,4],269:[7,2,5],270:[7,2,6],271:[7,2,7],272:[7,2,8],273:[7,3,4],274:[7,3,5],275:[7,3,6],276:[7,3,7],277:[7,3,8],278:[7,4,5],279:[7,4,6],280:[7,4,7],281:[7,4,8],282:[7,5,6],283:[7,5,7],284:[7,5,8],285:[7,6,7],286:[7,6,8],287:[7,7,8],288:[8,0,1],289:[8,0,2],290:[8,0,3],291:[8,0,4],292:[8,0,5],293:[8,0,6],294:[8,0,7],295:[8,0,8],296:[8,1,2],297:[8,1,3],298:[8,1,4],299:[8,1,5],300:[8,1,6],301:[8,1,7],302:[8,1,8],303:[8,2,3],304:[8,2,4],305:[8,2,5],306:[8,2,6],307:[8,2,7],308:[8,2,8],309:[8,3,4],310:[8,3,5],311:[8,3,6],312:[8,3,7],313:[8,3,8],314:[8,4,5],315:[8,4,6],316:[8,4,7],317:[8,4,8],318:[8,5,6],319:[8,5,7],320:[8,5,8],321:[8,6,7],322:[8,6,8],323:[8,7,8]}


#### Sudoku Example. 
Here, the sudoku problem is provided as input file

In [3]:
input_file=open('Sudoku_Input_File.txt','r')
input_suko=input_file.readlines()

dicta=ast.literal_eval(input_suko[0].replace('\n','').replace(' ',''))
dictb=ast.literal_eval(input_suko[1].replace('\n','').replace(' ',''))
dictc=ast.literal_eval(input_suko[2].replace('\n','').replace(' ',''))
dictd=ast.literal_eval(input_suko[3].replace('\n','').replace(' ',''))
dicte=ast.literal_eval(input_suko[4].replace('\n','').replace(' ',''))
dictf=ast.literal_eval(input_suko[5].replace('\n','').replace(' ',''))
dictg=ast.literal_eval(input_suko[6].replace('\n','').replace(' ',''))
dicth=ast.literal_eval(input_suko[7].replace('\n','').replace(' ',''))
dicti=ast.literal_eval(input_suko[8].replace('\n','').replace(' ',''))

#### Default Sudoku 
The below list variables - a1,b1,c1,d1,e1,f1,g1,h1,i1 represent rows of default sudoku. 
The below piece of code would convert it or rather rearrange the numbers across the rows as per the sudoku problem.

In [4]:
a1=[1,2,3,4,5,6,7,8,9]
b1=[4,5,6,7,8,9,1,2,3]
c1=[7,8,9,1,2,3,4,5,6]
d1=[9,1,2,3,4,5,6,7,8]
e1=[6,7,8,9,1,2,3,4,5]
f1=[3,4,5,6,7,8,9,1,2]
g1=[8,9,1,2,3,4,5,6,7]
h1=[5,6,7,8,9,1,2,3,4]
i1=[2,3,4,5,6,7,8,9,1]

def mod_def_list(a1input,dictainput):
    for a1key in dictainput.keys():
        x=a1input[a1key]
        a1index=a1input.index(dictainput[a1key])
        a1input[a1key]=dictainput[a1key]
        a1input[a1index]=x
    return(a1input)

a1mod=mod_def_list(a1,dicta)
b1mod=mod_def_list(b1,dictb)
c1mod=mod_def_list(c1,dictc)
d1mod=mod_def_list(d1,dictd)
e1mod=mod_def_list(e1,dicte)
f1mod=mod_def_list(f1,dictf)
g1mod=mod_def_list(g1,dictg)
h1mod=mod_def_list(h1,dicth)
i1mod=mod_def_list(i1,dicti)
sudo_input=[]
sudo_input.append(a1mod)
sudo_input.append(b1mod)
sudo_input.append(c1mod)
sudo_input.append(d1mod)
sudo_input.append(e1mod)
sudo_input.append(f1mod)
sudo_input.append(g1mod)
sudo_input.append(h1mod)
sudo_input.append(i1mod)

#### Initial Sudoku State
This state would consists of certain number of invalid numbers which would be the initial state. The objective would be to have the numbers across the rows swapped till the number of invalids become 0

In [5]:
sudo_input

[[1, 2, 3, 4, 9, 6, 7, 8, 5],
 [4, 3, 9, 7, 8, 6, 1, 2, 5],
 [1, 8, 5, 7, 3, 2, 4, 6, 9],
 [3, 4, 7, 9, 1, 5, 8, 2, 6],
 [7, 1, 8, 9, 6, 4, 3, 2, 5],
 [9, 1, 5, 6, 8, 3, 7, 4, 2],
 [7, 5, 2, 1, 3, 9, 4, 6, 8],
 [8, 3, 7, 5, 9, 2, 1, 6, 4],
 [6, 2, 4, 5, 3, 7, 8, 9, 1]]

In [6]:
items_to_be_validated={0:[],1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[]}
for q in range(9):
    if(q not in dicta.keys()):
        items_to_be_validated[0].append(q)
    if(q not in dictb.keys()):
        items_to_be_validated[1].append(q)
    if(q not in dictc.keys()):
        items_to_be_validated[2].append(q)
    if(q not in dictd.keys()):
        items_to_be_validated[3].append(q)
    if(q not in dicte.keys()):
        items_to_be_validated[4].append(q)
    if(q not in dictf.keys()):
        items_to_be_validated[5].append(q)
    if(q not in dictg.keys()):
        items_to_be_validated[6].append(q)
    if(q not in dicth.keys()):
        items_to_be_validated[7].append(q)
    if(q not in dicti.keys()):
        items_to_be_validated[8].append(q)

#items_to_be_validatedidated

#### Functions to calculate reward 

In [7]:
def calc_rewards(sudo,act_input):
    s=actions_dict[act_input]
    sudocopy=copy.deepcopy(sudo)
    if(s[1] in items_to_be_validated[s[0]] and s[2] in items_to_be_validated[s[0]]):
        x=sudocopy[s[0]][s[1]]
        y=sudocopy[s[0]][s[2]]
        sudocopy[s[0]][s[1]]=y
        sudocopy[s[0]][s[2]]=x
        a,b=get_rewards(sudocopy)
        action_reward=a-b
    else:
        action_reward=-1000
    return(action_reward)
    
def get_rewards(su):
    valid_invalid_list=['V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V','V']
    block1=[su[0][0],su[0][1],su[0][2],su[1][0],su[1][1],su[1][2],su[2][0],su[2][1],su[2][2]]
    block2=[su[0][3],su[0][4],su[0][5],su[1][3],su[1][4],su[1][5],su[2][3],su[2][4],su[2][5]]
    block3=[su[0][6],su[0][7],su[0][8],su[1][6],su[1][7],su[1][8],su[2][6],su[2][7],su[2][8]]
    block4=[su[3][0],su[3][1],su[3][2],su[4][0],su[4][1],su[4][2],su[5][0],su[5][1],su[5][2]]
    block5=[su[3][3],su[3][4],su[3][5],su[4][3],su[4][4],su[4][5],su[5][3],su[5][4],su[5][5]]
    block6=[su[3][6],su[3][7],su[3][8],su[4][6],su[4][7],su[4][8],su[5][6],su[5][7],su[5][8]]
    block7=[su[6][0],su[6][1],su[6][2],su[7][0],su[7][1],su[7][2],su[8][0],su[8][1],su[8][2]]
    block8=[su[6][3],su[6][4],su[6][5],su[7][3],su[7][4],su[7][5],su[8][3],su[8][4],su[8][5]]
    block9=[su[6][6],su[6][7],su[6][8],su[7][6],su[7][7],su[7][8],su[8][6],su[8][7],su[8][8]]

    #items_to_be_validated ={0:[1,2,3,4,5], 1:[1,2,3,4,5]}
    sucount=0
    for i in su: 
        validation_items=items_to_be_validated[sucount]
        for item in validation_items:
            if(item in [0,1,2] and sucount<3):
                if(block1.count(i[item]) > 1):
                    valid_invalid_list[item + sucount*(len(i))]='IV'
        
            if(item in [3,4,5] and sucount<3):
                if(block2.count(i[item]) > 1):
                    valid_invalid_list[item+ sucount*(len(i))]='IV'
            if(item in [6,7,8] and sucount<3):
                if(block3.count(i[item]) > 1):
                    valid_invalid_list[item+ sucount*(len(i))]='IV'

            if(item in [0,1,2] and sucount>2 and sucount<6):
                if(block4.count(i[item]) > 1):
                    valid_invalid_list[item + sucount*(len(i))]='IV'
            if(item in [3,4,5] and sucount>2 and sucount<6):
                if(block5.count(i[item]) > 1):
                    valid_invalid_list[item+ sucount*(len(i))]='IV'
            if(item in [6,7,8] and sucount>2 and sucount<6):
                if(block6.count(i[item]) > 1):
                    valid_invalid_list[item+ sucount*(len(i))]='IV'

            if(item in [0,1,2] and sucount>5 and sucount<9):
                if(block7.count(i[item]) > 1):
                    valid_invalid_list[item + sucount*(len(i))]='IV'
            if(item in [3,4,5] and sucount>5 and sucount<9):
                if(block8.count(i[item]) > 1):
                    valid_invalid_list[item+ sucount*(len(i))]='IV'
            if(item in [6,7,8] and sucount>5 and sucount<9):
                if(block9.count(i[item]) > 1):
                    valid_invalid_list[item+ sucount*(len(i))]='IV'
            if([su[0][item],su[1][item],su[2][item],su[3][item],su[4][item],su[5][item],su[6][item],su[7][item],su[8][item]].count(i[item])>1):
                valid_invalid_list[item+ sucount*(len(i))]='IV'
            if(i.count(i[item])>1):
                valid_invalid_list[item+ sucount*(len(i))]='IV'


        sucount+=1
    return(valid_invalid_list.count('V'),valid_invalid_list.count('IV'))

#### Function to execute the selected action

In [8]:
def exec_modify(llist,ldictitem):
    x=llist[ldictitem[0]][ldictitem[1]]
    y=llist[ldictitem[0]][ldictitem[2]]
    llist[ldictitem[0]][ldictitem[1]]=y
    llist[ldictitem[0]][ldictitem[2]]=x
    return(llist)

#### Iteration Loop
Successive runs of 2000 steps till convergence is reached

In [9]:
#su_input.count('IV')>0
V=[]
c,d=get_rewards(sudo_input)
prev_action_tried=[]
sudo_input_orig=copy.deepcopy(sudo_input)
counter=1
while(d>0):
    if(counter < 2001):
        V=[]
        for act in actions_dict.keys():
            #print(str(state)+' : ' + str(calc_rewards(su_input,state)))
            V.append(calc_rewards(sudo_input,act))
        Vmax=max(V)
        Vmaxindexcheck=V.index(Vmax)
        if(actions_dict[Vmaxindexcheck]==prev_action_tried and len(prev_action_tried)>0):
            if(V.count(Vmax)>1):
                indexmaxvalues=[maxindex for maxindex in range(len(V)) if V[maxindex]==Vmax]
                Vmaxindex=random.choice(indexmaxvalues)
            else:
                indexnonzerovalues=[maxindex for maxindex in range(len(V)) if V[maxindex]>0]
                Vmaxindex=random.choice(indexnonzerovalues)
                if(actions_dict[Vmaxindex]==prev_action_tried and len(prev_action_tried)>0 and len(indexnonzerovalues)==1):
                    print('Locked State')
                    break
                #else:
                #    continue
        else:
            Vmaxindex=V.index(Vmax)
    
    
        sudo_input=exec_modify(sudo_input,actions_dict[Vmaxindex])
        c,d=get_rewards(sudo_input)
        prev_action_tried=actions_dict[Vmaxindex]
        #print(str(actions_dict[Vmaxindex]) + ' - ' + str(d) + '-'+str(counter))
        counter=counter+1
    else:
        print('No solution in 2000 steps. Need to Starting again')
        print('\n')
        counter=1
        sudo_input=copy.deepcopy(sudo_input_orig)

print('Solved - '+str(counter) +'\n')
print(sudo_input)



No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution in 2000 steps. Need to Starting again


No solution 

### Solved Sudoku

In [10]:
sudo_input

[[6, 7, 2, 3, 9, 1, 5, 8, 4],
 [4, 3, 9, 8, 5, 6, 2, 7, 1],
 [1, 8, 5, 7, 4, 2, 6, 3, 9],
 [2, 4, 7, 9, 8, 5, 3, 1, 6],
 [8, 1, 3, 4, 6, 7, 9, 2, 5],
 [9, 5, 6, 2, 1, 3, 7, 4, 8],
 [7, 6, 8, 1, 2, 9, 4, 5, 3],
 [3, 9, 4, 5, 7, 8, 1, 6, 2],
 [5, 2, 1, 6, 3, 4, 8, 9, 7]]